## Vi testar vår model på en video stream
Vi börjar med att installera yt_dlp med pip

In [5]:
%pip install yt_dlp
%pip install deep-sort-realtime


^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Sedan kör vi på youtube video

In [1]:
import sys
import os

# Add the 'src' directory to the Python path
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Levi Main
video_url = 'https://www.youtube.com/watch?v=Wr9b5aYA4mI'

# Model to use
model_path = 'output/yolo_model_new/weights/best.pt'

from src.counter import count_classes

# Function to count the skiers and snowboarders in the youtube video
count_classes(video_url, model_path, 820)

[youtube] Extracting URL: https://www.youtube.com/watch?v=Wr9b5aYA4mI
[youtube] Wr9b5aYA4mI: Downloading webpage
[youtube] Wr9b5aYA4mI: Downloading tv client config
[youtube] Wr9b5aYA4mI: Downloading player 7d1d50a6
[youtube] Wr9b5aYA4mI: Downloading tv player API JSON
[youtube] Wr9b5aYA4mI: Downloading ios player API JSON
[youtube] Wr9b5aYA4mI: Downloading m3u8 information
[youtube] Wr9b5aYA4mI: Downloading m3u8 information

0: 384x640 (no detections), 42.0ms
Speed: 4.8ms preprocess, 42.0ms inference, 19.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.3ms
Speed: 3.7ms preprocess, 11.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.4ms
Speed: 2.3ms preprocess, 11.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.3ms
Speed: 1.9ms preprocess, 12.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.9ms
Sp